In [4]:
# Importing CSV files and display them as dataframes
import numpy as np
import pandas as pd
rating_dataset = pd.read_csv("ratings.csv")
rating_dataset = rating_dataset.drop(columns =['timestamp'])
rating_dataset.head()
movie_dataset = pd.read_csv("movies.csv")
#movie_dataset.head()

 
data = pd.merge(rating_dataset,movie_dataset, left_on='movieId',right_on ='movieId',how='left')
data= data[:5]

data.head()

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [5]:
# check if there is any data in any column missing or not available
data.isnull().values.any()

False

False

In [6]:
# find threshold by averging the movie ratings
# get only top5 rated movies per user,threshold is 4.5
maxRating = data.loc[data['rating'].idxmax()]['rating'] 
toprated_user_movies = data.loc[data['rating'] >= maxRating/2+1]
toprated_user_movies = toprated_user_movies.sort_values(['userId','rating'],ascending=[True,False])
toprated_user_movies.reset_index(inplace=True, drop=True)
toprated_user_movies.head()
# checking if user has rated a movie multiple times
#toprated_user_movies.duplicated(subset=['userId','movieId'])
# actual data is 20000263 after this step it is 646297

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [7]:
# Break up the big genre string into a string array
toprated_user_movies['genres'] = toprated_user_movies['genres'].str.split('|')
# Convert genres to string value
toprated_user_movies['genres'] = np.array(toprated_user_movies['genres'])
toprated_user_movies.head()

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),"[Adventure, Children, Fantasy]"
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...","[Adventure, Drama, Fantasy, Mystery, Sci-Fi]"
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"[Mystery, Sci-Fi, Thriller]"
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]"
4,1,50,3.5,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]"


,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),"[Adventure, Children, Fantasy]"
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...","[Adventure, Drama, Fantasy, Mystery, Sci-Fi]"
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"[Mystery, Sci-Fi, Thriller]"
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]"
4,1,50,3.5,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]"


In [8]:
#target domain
books = pd.read_csv('book_data.csv')
books = books[['book_title','genres','book_rating']]
books = books.dropna(how='any', axis=0)
# Break up the big genre string into a string array
books['genres'] = books['genres'].str.split('|')
# Convert genres to string value
books['genres'] = np.array(books['genres'])

books['length'] = books['genres'].str.len()
books = books.sort_values(['length'])
books = books.drop(columns = ['length'])
books.reset_index(inplace=True, drop=True)
books.head()

,book_title,genres,book_rating
0,The Caine Batter,[Literature],3.20
1,الأيام,[Biography],3.53
2,Biological Big Bang: Panspermia and the Origin...,[Nonfiction],4.50
3,Mugglenet.Com's What Will Happen in Harry Pott...,[Nonfiction],4.20
4,"Crónica del rey cautivo (El último Qassatar, #1)",[Fantasy],4.13


,book_title,genres,book_rating
0,The Caine Batter,[Literature],3.20
1,الأيام,[Biography],3.53
2,Biological Big Bang: Panspermia and the Origin...,[Nonfiction],4.50
3,Mugglenet.Com's What Will Happen in Harry Pott...,[Nonfiction],4.20
4,"Crónica del rey cautivo (El último Qassatar, #1)",[Fantasy],4.13


In [14]:
def book_basedon_movie(genre):
    # finding intersecti:on and recommending books
    data = []
   
    book_titles = []
    scores = []
    ratings = []
    for i,row in books.iterrows():
        movies_set = set(genre)
        books_set = set(row.genres)
        intersection = movies_set.intersection(books_set)
        book_titles.append(row.book_title)
        ratings.append(row.book_rating)
        scores.append(float(len(intersection))/(len(movies_set)+len(books_set)))
    final = pd.DataFrame(columns = ['book_title','score','ratings'])
    final['book_title'] = book_titles
    final['score'] = scores
    final['ratings'] = ratings
    return final

books_array = []
for i, row in toprated_user_movies.iterrows():
    book_df = book_basedon_movie(row.genres)
    book_df = book_df.loc[book_df['ratings'] >= 3]
    book_df = book_df.sort_values(['score','ratings'],ascending=[False,False]).head(5)
    books_array.append(','.join([str(elem) for elem in book_df['book_title']]))

toprated_user_movies['books'] = books_array

toprated_user_movies.head()




                                             book_title  score  ratings
3890  Selections from Harry Potter and the Order of ...    0.4     4.60
3408                               Jeweler's Apprentice    0.4     4.13
3206                                         Waterwight    0.4     3.94
3392                             Heritage of the Xandim    0.4     3.79
3630                                     Captain Sinbad    0.4     3.58
                                             book_title  score  ratings
3890  Selections from Harry Potter and the Order of ...    0.4     4.60
3408                               Jeweler's Apprentice    0.4     4.13
3206                                         Waterwight    0.4     3.94
3392                             Heritage of the Xandim    0.4     3.79
3630                                     Captain Sinbad    0.4     3.58
                                       book_title     score  ratings
12844            The 39 Clues Complete Collection  0.333333     4.3

,userId,movieId,rating,title,genres,books
0,1,2,3.5,Jumanji (1995),"[Adventure, Children, Fantasy]",Selections from Harry Potter and the Order of ...
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...","[Adventure, Drama, Fantasy, Mystery, Sci-Fi]","The 39 Clues Complete Collection,This Book Is ..."
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"[Mystery, Sci-Fi, Thriller]","Shannon's Land,Rain of Terror,Repercussions,Do..."
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]","Shannon's Land,Rain of Terror,Repercussions,Do..."
4,1,50,3.5,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]","Dead Souls,Blood Lines,Deadly Game,Der Augenjä..."


,userId,movieId,rating,title,genres,books
0,1,2,3.5,Jumanji (1995),"[Adventure, Children, Fantasy]",Selections from Harry Potter and the Order of ...
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...","[Adventure, Drama, Fantasy, Mystery, Sci-Fi]","The 39 Clues Complete Collection,This Book Is ..."
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"[Mystery, Sci-Fi, Thriller]","Shannon's Land,Rain of Terror,Repercussions,Do..."
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]","Shannon's Land,Rain of Terror,Repercussions,Do..."
4,1,50,3.5,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]","Dead Souls,Blood Lines,Deadly Game,Der Augenjä..."
